### 라이브러리 호출

In [ ]:
# 연산 처리  패키지
import pandas as pd
import numpy as np

# 시각화 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# 통계분석 패키지
import statsmodels.api as sm

# 그래프 설정
%matplotlib inline

# 경고 메세지 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, roc_curve, mean_squared_error, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder
import math
# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

In [ ]:
# 코랩 mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 모델 함수

In [ ]:
# 로지스틱 회귀 함수
def logistic_classifier(X, y):

    # 평가지표 / best param 저장 list
    acc = []
    prec = []
    recall = []
    auc = []

    acc_opt = []
    prec_opt = []
    recall_opt = []

    opt_thres = []

    # LogisticRegression 초기화
    model = LogisticRegression()

    for i in range(100):

        # y 범주 비율에 맞춰 train-test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

        # 데이터 Random Sampling
        X_train_rd = X_train.sample(n = 10000)
        y_train_rd = y_train[X_train_rd.index]

        # SMOTE로 X oversampling
        smt = SMOTE(sampling_strategy = 'auto')
        X_train_sm, y_train_sm = smt.fit_resample(X_train_rd, y_train_rd)

        # 모델 학습 / 예측
        model.fit(X_train_sm, y_train_sm)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:,1] # 양성 클래스에 대한 확률 추출

        # thr = 0.5일 때 평가지표 저장
        acc.append((accuracy_score(y_test, y_pred)))
        prec.append(precision_score(y_test, y_pred))
        recall.append(recall_score(y_test, y_pred))
        auc.append(roc_auc_score(y_test, y_proba))

        # Optimal thr 저장
        fper, tper, thresholds = roc_curve(y_test, y_proba)
        optimal_idx = np.argmax(tper - fper)   # fpr, tpr 간 차이가 가장 클 때의 index 저장

        # Optimal thr일 때 평가지표 저장
        y_optpred = (y_proba > thresholds[optimal_idx]).astype(int)

        acc_opt.append(accuracy_score(y_test, y_optpred))
        prec_opt.append(precision_score(y_test, y_optpred))
        recall_opt.append(recall_score(y_test, y_optpred))
        opt_thres.append(thresholds[optimal_idx])

        # 횟수 출력
        if((i+1)%10 == 0) :
            print(f"{i+1}번째 완료")

    # 결과 DataFrame 생성
    results = pd.DataFrame({
        '0.5 acc' : acc,
        '0.5 prec': prec,
        '0.5 recall': recall,
        'opt acc' : acc_opt,
        'opt prec' : prec_opt,
        'opt recall' : recall_opt,
        'auc': auc,
        'opt threshold' : opt_thres
    })

    return results

### Logistic Regression 결과

In [ ]:
# base preprocess 데이터로부터 y값 읽어오기
base = pd.read_csv("/content/drive/MyDrive/기계학습의이해/Dataset/base_process.csv", encoding = "euc-kr", engine='python')
yn_y = base['사고유무']

In [ ]:
yn_X = pd.read_csv(f"/content/drive/MyDrive/기계학습의이해/Dataset/DF/1111211.csv", encoding = 'euc-kr', engine='python')
predicted = pd.read_csv(f"/content/drive/MyDrive/기계학습의이해/Dataset/predicted_result.csv", encoding = 'UTF8', engine='python')

In [ ]:
yn_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210315 entries, 0 to 210314
Data columns (total 43 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   여성            210315 non-null  bool 
 1   외산            210315 non-null  bool 
 2   1인 및 지정1인     210315 non-null  bool 
 3   가족 및 지정1인     210315 non-null  bool 
 4   가족및형제자매한정     210315 non-null  bool 
 5   가족한정(형제자매제외)  210315 non-null  bool 
 6   기명피보험자1인한정    210315 non-null  bool 
 7   누구나(기본)       210315 non-null  bool 
 8   부부 및 지정1인     210315 non-null  bool 
 9   부부한정          210315 non-null  bool 
 10  임직원한정         210315 non-null  bool 
 11  미가입           210315 non-null  bool 
 12  20.0          210315 non-null  bool 
 13  30.0          210315 non-null  bool 
 14  40.0          210315 non-null  bool 
 15  50.0          210315 non-null  bool 
 16  60.0          210315 non-null  bool 
 17  70.0          210315 non-null  bool 
 18  80.0          210315 non-null  bool 
 19  90

### 파생변수 추가

In [ ]:
# 연령대별 사고율
yn_X['연령대사고율'] = np.where(yn_X['30.0'] == True, predicted['30대'],
                        np.where(yn_X['40.0'] == True, predicted['40대'],
                        np.where(yn_X['50.0'] == True, predicted['50대'],
                        np.where(yn_X['60.0'] == True, predicted['~64'],
                        np.where(yn_X['20.0'] == True, predicted['20대'],
                        np.where(yn_X['70.0'] == True, predicted['65~'],
                        np.where(yn_X['80.0'] == True, predicted['65~'],
                        np.where(yn_X['90.0'] == True, predicted['65~'], predicted['20대'])))))))) # 10대가 지워짐


# 성별별 사고율
yn_X['성별사고율'] = np.where(yn_X['여성'] == False, predicted['남성'], predicted['여성'])

# 특약별 사고율
yn_X['특약사고율'] = np.where((yn_X['기명피보험자1인한정'] == True) | (yn_X['1인 및 지정1인'] == True), predicted['특약 기명피보험자1인'],
                        np.where((yn_X['부부한정'] == True) | (yn_X['부부 및 지정1인'] == True), predicted['특약부부운전'],
                        np.where((yn_X['가족한정(형제자매제외)'] == True) | (yn_X['가족및형제자매한정'] == True) | (yn_X['가족 및 지정1인'] == True), predicted['특약 가족'],
                        np.where(yn_X['누구나(기본)'] == True, predicted['특약 미가입'],
                        np.where(yn_X['임직원한정'] == True, predicted['특약 기타'], predicted['특약부부운전']))))) # 부부및자녀한정이지워짐
yn_X.head(5)

,여성,외산,1인 및 지정1인,가족 및 지정1인,가족및형제자매한정,가족한정(형제자매제외),기명피보험자1인한정,누구나(기본),부부 및 지정1인,부부한정,...,5.0,6.0,7.0,8.0,1억이하,5천만원이하,미가입.1,연령대사고율,성별사고율,특약사고율
0,True,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,True,False,0.000596,0.022789,0.062796
1,True,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,True,False,0.000596,0.022789,0.056164
2,True,False,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,0.000596,0.022789,0.062796
3,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,0.000596,0.022789,0.062796
4,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,0.000596,0.022789,0.062796


In [ ]:
X1 = yn_X.drop(columns = ['연령대사고율', '성별사고율', '특약사고율'], axis = 1)
X2 = yn_X.drop(columns = ['성별사고율', '특약사고율'], axis = 1)
X3 = yn_X.drop(columns = ['연령대사고율', '특약사고율'], axis = 1)
X4 = yn_X.drop(columns = ['연령대사고율', '성별사고율'], axis = 1)
X5 = yn_X.drop(columns = ['특약사고율'], axis = 1)
X6 = yn_X.drop(columns = ['성별사고율'], axis = 1)
X7 = yn_X.drop(columns = ['연령대사고율'], axis = 1)
X8 = yn_X

### 모델 실행

In [ ]:
# Logistic Classifier model fit

total = pd.DataFrame()

# X 지정
yn_X = X1

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '000'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료


In [ ]:
# X 지정
yn_X = X2

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '100'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

################
# X 지정
yn_X = X3

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '010'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

################
# X 지정
yn_X = X4

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '001'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

###############
# X 지정
yn_X = X5

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '110'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

################
# X 지정
yn_X = X6

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '101'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

##################
# X 지정
yn_X = X7

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '011'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")

###############################
# X 지정
yn_X = X8

# smote 계산을 위해 data type float로 변경
for col_name in yn_X.columns:
    yn_X[col_name] = yn_X[col_name].astype(float)

# 모델 실행
logistic_rst = logistic_classifier(yn_X, yn_y)

# csv 전체 합쳐서 만드는 코드
logistic_rst['파생변수'] = '111'
total = pd.concat([total, logistic_rst])

total.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", index = False, encoding="euc-kr")


10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료
10번째 완료
20번째 완료
30번째 완료
40번째 완료
50번째 완료
60번째 완료
70번째 완료
80번째 완료
90번째 완료
100번째 완료


### charting

In [ ]:
rst = pd.read_csv("/content/drive/MyDrive/기계학습의이해/LR/LR_2nd_result.csv", encoding = "euc-kr", engine='python')
rst

,0.5 acc,0.5 prec,0.5 recall,opt acc,opt prec,opt recall,auc,opt threshold,파생변수
0,0.597223,0.218246,0.543031,0.586454,0.216875,0.561605,0.605048,0.493961,0
1,0.590709,0.214410,0.540905,0.523857,0.206594,0.647668,0.605065,0.460195,0
2,0.594347,0.218694,0.551680,0.558710,0.214601,0.613498,0.607924,0.480475,0
3,0.579821,0.214689,0.566567,0.583434,0.215686,0.563023,0.602728,0.501530,0
4,0.598531,0.218449,0.540905,0.541616,0.210402,0.629803,0.606034,0.470663,0
...,...,...,...,...,...,...,...,...,...
795,0.599078,0.216756,0.532256,0.550626,0.209711,0.606834,0.603664,0.475227,111
796,0.599719,0.216898,0.531405,0.565128,0.213042,0.591521,0.604169,0.481405,111
797,0.595107,0.219939,0.555508,0.571333,0.216565,0.594641,0.607210,0.487086,111
798,0.596772,0.218555,0.545442,0.584576,0.216559,0.564441,0.604885,0.492956,111


In [ ]:
# charting : 1

# 결과값 top 50 비교
print("======= 50개 =======")
# opt acc 기준으로 내림차순 정렬 후 상위 10개 row 선택
results_sorted = rst.sort_values(by='opt acc', ascending=False).head(50)
results_sorted['파생변수'].value_counts()

======= 50개 =======


파생변수
10     9
1      9
11     7
101    6
0      6
100    5
110    4
111    4
Name: count, dtype: int64

In [ ]:
# 결과값 top 100 비교
print("======= 100개 =======")
# opt acc 기준으로 내림차순 정렬 후 상위 100개 row 선택
results_sorted = rst.sort_values(by='opt acc', ascending=False).head(100)
results_sorted['파생변수'].value_counts()

======= 100개 =======


파생변수
1      19
10     15
101    15
11     13
0      11
111    11
110     9
100     7
Name: count, dtype: int64

In [ ]:
# charting : 2
# 결과값 평균 비교
rst_accmean = rst.groupby('파생변수')['opt acc'].mean()
rst_accmean.sort_values(ascending = False)

파생변수
1      0.569138
11     0.567796
101    0.567529
10     0.566398
0      0.564146
111    0.563586
100    0.562598
110    0.561312
Name: opt acc, dtype: float64

In [ ]:
# 결과값 중앙값 비교
rst_accmedian = rst.groupby('파생변수')['opt acc'].median()
rst_accmedian.sort_values(ascending = False)

파생변수
11     0.571702
1      0.569764
101    0.567482
111    0.565925
10     0.563631
110    0.561871
100    0.560932
0      0.559886
Name: opt acc, dtype: float64

### 결론

1111211 데이터는 특약 사고율을 추가해 모델링

In [ ]:
# 파생변수 전처리 저장
X4.to_csv("/content/drive/MyDrive/기계학습의이해/LR/LR1111211_001.csv", index = False, encoding="euc-kr")